In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [ ]:
sample_output = pd.read_csv('/kaggle/input/nlp-getting-started/sample_submission.csv')
train_data = pd.read_csv('/kaggle/input/nlp-getting-started/train.csv')
test_data = pd.read_csv('/kaggle/input/nlp-getting-started/test.csv')

In [ ]:
sample_output.head()

In [ ]:
train_data.head()

In [ ]:
train_data.shape

In [ ]:
train_data.isnull().sum()

In [ ]:
train_data.isnull().sum()

In [ ]:
train_data.shape

In [ ]:
import re
def  clean_text(df, text_field):
    df[text_field] = df[text_field].str.lower()
    df[text_field] = df[text_field].apply(lambda elem: re.sub(r"(@[A-Za-z0-9]+)|([^0-9A-Za-z \t])|(\w+:\/\/\S+)|^rt|http.+?", "", elem))  
    return df
test_clean = clean_text(test_data, "text")
train_clean = clean_text(train_data, "text")

In [ ]:
#To have fair training i.e avoid overfitting/underfitting we do something to make sure number of real tweets is almost same in number as that of fake ones!
from sklearn.utils import resample
train_majority = train_clean[train_clean.target==0]
train_minority = train_clean[train_clean.target==1]
train_minority_upsampled = resample(train_minority, 
                                 replace=True,    
                                 n_samples=len(train_majority),   
                                 random_state=123)
train_upsampled = pd.concat([train_minority_upsampled, train_majority])
train_upsampled['target'].value_counts()

In [ ]:
from sklearn.pipeline import Pipeline

In [ ]:
from sklearn.svm import LinearSVC

In [ ]:
from sklearn.metrics import classification_report,confusion_matrix,accuracy_score
from sklearn.feature_extraction.text import TfidfVectorizer

In [ ]:
train_upsampled.shape

In [ ]:
test_clean.shape

In [ ]:
X_train = train_upsampled['text']
Y_train = train_upsampled['target']

In [ ]:
clf = Pipeline([('tfidf',TfidfVectorizer()),('SupportVectorMachine',LinearSVC())])

In [ ]:
clf

In [ ]:
clf.fit(X_train,Y_train)

In [ ]:
test_clean.head()

In [ ]:
X_test = test_clean['text']

In [ ]:
predictions = clf.predict(X_test)

In [ ]:
sample_output.head()

In [ ]:
output = pd.DataFrame({'id': test_clean.id, 'target': predictions})
output.to_csv('my_submission.csv', index=False)
print("Your submission was successfully saved!")

In [ ]:
pd.read_csv('my_submission.csv').head()